<a href="https://colab.research.google.com/github/omkar109/Sleep-Disorder-ML-Model/blob/main/Sleep_Disorder_ML_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import pandas as pd

#Hyperparameters
learning_rate = 0.01
batch_size = 12
epochs = 100
classification_threshold = 0.8

data = pd.read_csv("Sleep_health_and_lifestyle_dataset.csv")
#data = data.reindex(np.random.permutation(data.index)) #Shuffles the dataset
data = data.sample(frac=1).reset_index(drop=True) #Shuffles the dataset
#train_data = data.sample(frac=0.8)
#test_data = data.drop(train_data.index)
#print(train_data["Occupation"].unique().tolist())
#print(train_data.describe(include="all"))

def one_hot_engineering(feature_list, data):
  for feature in feature_list:
    vocab = data[feature].unique().tolist()
    #print(data[feature])
    #print(vocab)
    preprocessing_layer = tf.keras.layers.StringLookup(vocabulary=vocab)
    #print(data[feature])
    temp_feature = preprocessing_layer(tf.convert_to_tensor(data[feature])).numpy() -1.0
    prelayer_2 = tf.keras.layers.CategoryEncoding(num_tokens=len(vocab),output_mode="one_hot")
    one_hot_array = prelayer_2(tf.convert_to_tensor(temp_feature)).numpy()
    new_df = pd.DataFrame(one_hot_array, columns=[f'{category}' for category in vocab])
    #print(new_df)
    #print(data)
    data = pd.concat([data, new_df], axis=1)
    #print(data)
    #print(data["Occupation"])
  return data

def label_engineering(data):
  data["Normal"] = data["Normal"] + data["Normal Weight"]
  data = data.drop("Normal Weight", axis=1)
  data["Has Sleep Disorder"] = (data["Sleep Disorder"] != "None").astype(float)
  return data

feature_list = ["Gender", "Occupation", "BMI Category"]
data = one_hot_engineering(feature_list, data)
#test_data = one_hot_engineering(feature_list, test_data)

data = label_engineering(data)
#test_data = label_engineering(test_data)

#inputs = keras.Input(shape=(23,)) #Our ML model is inputted 23 features
#dense_layer_1 = layers.Dense(1,activation=None) #One dense layer, has one output, no activation function
#x = dense_layer_1(inputs) #This passes inputs into dense_layer_1, and x is the output

#model = keras.Model(inputs=inputs, outputs=x)

#Scale the features using normalization
def normalize_features(data):
  data_normalized = (data - data.mean()) / data.std()
  data_normalized = data_normalized.drop("Has Sleep Disorder", axis=1)
  data_normalized = pd.concat([data_normalized, data["Has Sleep Disorder"]], axis=1)
  return data_normalized

data = normalize_features(data)

#We need to scale data and preprocess the categorical data before this step
#features = {name:np.array(value) for name, value in data_normalized.items()}
#del features["Person ID"]
#del features["Gender"]
#del features["Occupation"]
#del features["BMI Category"]
#del features["Blood Pressure"]
#label = np.array(features.pop("Has Sleep Disorder"))

def feed_to_model(delete_features):
  features = {name:np.array(value) for name, value in data.items()}
  label = np.array(features.pop("Has Sleep Disorder"))
  for feature in delete_features:
    del features[feature]
  return features, label

delete_features = ["Sleep Disorder", "Occupation", "Gender", "Blood Pressure", "BMI Category"]
features, label = feed_to_model(delete_features)

#Make this less hardcoded later
inputs = {
    "Age": tf.keras.Input(shape=(1,),dtype="float64"),
    "Sleep Duration" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Quality of Sleep" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Physical Activity Level" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Stress Level" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Heart Rate" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Daily Steps" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Male" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Female" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Nurse" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Engineer" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Accountant" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Scientist" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Lawyer" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Salesperson" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Manager" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Overweight" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Normal" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Obese" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Doctor" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Software Engineer" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Teacher" : tf.keras.Input(shape=(1,),dtype="float64"),
    "Sales Representative" : tf.keras.Input(shape=(1,),dtype="float64")
}

concatenated_inputs = tf.keras.layers.Concatenate()(inputs.values())
dense = layers.Dense(units=1, input_shape=(1,),name='dense_layer', activation=tf.sigmoid)
dense_output = dense(concatenated_inputs)
"""Create and compile a simple classification model."""
outputs = {
  'dense': dense_output
}
model = tf.keras.Model(inputs=inputs, outputs=outputs)
metrics=[tf.keras.metrics.BinaryAccuracy(threshold=classification_threshold),
         tf.keras.metrics.Precision(thresholds=classification_threshold),
         tf.keras.metrics.Recall(thresholds=classification_threshold)]
model.compile(optimizer=tf.keras.optimizers.experimental.RMSprop(learning_rate=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=metrics)

#model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
#              loss="mean_squared_error",
#              metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(x=features, y=label, batch_size=batch_size, epochs=epochs, validation_split=0.2)

#Split the code into training data and validation data, then test on the validation dataset


#Later at the end, make the code less hardcoded and more flexible (so when you change a value, you don't
#have to change it everywhere) also organize the code and leave comments

Epoch 1/100


<ipython-input-68-49300976c6a9>:60: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_normalized = (data - data.mean()) / data.std()
<ipython-input-68-49300976c6a9>:60: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_normalized = (data - data.mean()) / data.std()
/usr/local/lib/python3.10/dist-packages/keras/engine/functional.py:639: UserWarning: Input dict contained keys ['Person ID'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)


25/25 [==============================] - 1s 18ms/step - loss: 0.6247 - binary_accuracy: 0.6020 - precision: 0.6471 - recall: 0.0887 - val_loss: 0.4058 - val_binary_accuracy: 0.7200 - val_precision: 1.0000 - val_recall: 0.3226
Epoch 2/100
25/25 [==============================] - 0s 4ms/step - loss: 0.3518 - binary_accuracy: 0.7191 - precision: 0.8704 - recall: 0.3790 - val_loss: 0.3564 - val_binary_accuracy: 0.8400 - val_precision: 0.9130 - val_recall: 0.6774
Epoch 3/100
25/25 [==============================] - 0s 4ms/step - loss: 0.2851 - binary_accuracy: 0.8796 - precision: 0.9074 - recall: 0.7903 - val_loss: 0.3681 - val_binary_accuracy: 0.8933 - val_precision: 0.8710 - val_recall: 0.8710
Epoch 4/100
25/25 [==============================] - 0s 4ms/step - loss: 0.2631 - binary_accuracy: 0.9365 - precision: 0.9200 - recall: 0.9274 - val_loss: 0.3898 - val_binary_accuracy: 0.8933 - val_precision: 0.8710 - val_recall: 0.8710
Epoch 5/100
25/25 [==============================] - 0s 4ms/ste